In [1]:
import scipy
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn import linear_model
import tensorflow
import ee
import json
from collections import OrderedDict
import time
import math
import csv
import os

# import custom module
import ALR_functions_server_side as alr

ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWgvWVDcgSxMC83OIaHjI2oYhzwbrZVK9DQmfr2wfb8d85Pl27_Gv8U



Successfully saved authorization token.


In [2]:
# order of bands from SL2P output:
# 00-11: 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 
# 12-19: 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 
# 20-26: 'QA10', 'QA20', 'QA60', 'date', 'cosVZA', 'cosSZA', 'cosRAA', 
# 27-32: 'estimateLAI', 'partition', 'networkID', 'errorLAI', 'partition_1', 'networkID_1'

outputName = 'LAI'

# define input image
testImage = ee.Image('users/kateharvey/scaled_image')
inputImage = ee.Image(testImage.select(1,2,3,7,22,23,27,28,29,30,31,32))
inputImage_bands = ee.List(['B2', 'B3', 'B4', 'B8', 'QA60', 'date', 'estimateLAI', 'partition', 'networkID', 'errorLAI', 'partition_1', 'networkID_1'])

# name bands of inputImage and scale response band
inputImage = inputImage.rename(inputImage_bands)

inputImage = inputImage.addBands(inputImage.select('estimateLAI').divide(1000), overwrite=True)

In [3]:
# Below we define a list of strings representing the expressions for each vegetation index as a function of the bands in the input image
# More vegetation indices can be defined, but the list CANNOT contain any two vegetation indices which are a linear combination of each
# other or LARs will fail to select the requested number of variables

# The formatting of the expression must be
# "<name of VI> = <expression with band names from inputImage_bands used as variables in the form b('<band name>')"

# Only include VIs that use 10 m bands (B2, B3, B4, B8)
input_VI_definition = ee.List(["RAW_B2  = b('B2')",
                               "RAW_B3  = b('B3')",
                               "RAW_B4  = b('B4')",
                               "RAW_B8  = b('B8')",
                               "GI      = b('B3')/b('B4')",
                             # "RVI3    = b('B4')/b('B6')",
                             # "SR3     = b('B5')/b('B4')",
                             # "GM1     = b('B6')/b('B3')",
                             # "GM2     = b('B6')/b('B5')",
                             # "SR2     = b('B7')/b('B3')",
                             # "PSSR    = b('B7')/b('B4')",
                               "SGI     = b('B8')/b('B4')",
                             # "MSI     = b('B11')/b('B7')",
                             # "II      = b('B11')/b('B12')",
                               "GVI     = (b('B8')/b('B3'))-1",
                             # "PSRI    = (b('B4')-b('B3'))/b('B6')",
                               "NDVI3   = ((b('B8')-b('B4'))/(b('B8')))+b('B4')",
                             # "SR5     = 1/b('B5')",
                             # "SR6     = b('B4')/(b('B3')*b('B5'))",
                             # "SR7     = b('B8')/(b('B3')*b('B5'))",
                             # "IPVI    = b('B7')/(b('B7')+b('B4'))",
                             # "ARI     = (1/b('B3'))-(1/b('B5'))",
                             # "ARI2    = b('B7')*((1/b('B3'))-(1/b('B5')))",
                               "NDVI    = (b('B8')-b('B4'))/(b('B8')+b('B4'))",
                               "GNDVI   = (b('B8')-b('B3'))/(b('B8')+b('B3'))",
                             # "NDWI    = (b('B8')-b('B11'))/(b('B8')+b('B11'))",
                             # "NDREVI  = (b('B8')-b('B5'))/(b('B8')+b('B5'))",
                               "NDGI    = (b('B3')-b('B4'))/(b('B3')+b('B4'))",
                             # "NDI1    = (b('B7')-b('B5'))/(b('B7')-b('B4'))",
                             # "NDI2    = (b('B8')-b('B5'))/(b('B8')-b('B4'))",
                             # "RENDVI  = (b('B6')-b('B5'))/(b('B6')+b('B5'))",
                             # "OSAVI   = (1.16*(b('B7')-b('B4')))/(b('B7')+b('B4')+0.61)",
                             # "NMDI    = (b('B8')-(b('B11')-b('B12')))/(b('B8')+(b('B11')-b('B12')))",
                             # "HI      = ((b('B3')-b('B5'))/(b('B3')+b('B5')))-0.5*b('B5')",
                             # "GVSP    = (-0.283*b('B3') - 0.66*b('B4') + 0.577*b('B6') + 0.388*b('B8'))/(0.433*b('B3') - 0.632*b('B4') + 0.586*b('B6') + 0.264*b('B8A'))",
                             # "MCARI   = ((b('B5')-b('B4'))-0.2*(b('B5')-b('B3')))*(b('B5')/b('B4'))",
                             # "TCARI   = 3*((b('B5')-b('B4'))-0.2*(b('B5')-b('B3'))*(b('B5')/b('B4')))",
                               "EVI     = 2.5*((b('B8')-b('B4'))/(b('B8')+6*b('B4')-7.5*b('B3')+1))",
                               "EVI2    = 2.5*((b('B8')-b('B4'))/(b('B8')+2.4*b('B4')+1))",
                               "RDVI    = (b('B8')-b('B4'))/((b('B8')+b('B4'))**0.5)",
                               "MSR     = ((b('B8')/b('B4'))-1)/((b('B8')/b('B4'))**0.5+1)",
                             # "MSAVI   = 0.5*(2*b('B7')+1-((2*b('B7')+1)**2-8*(b('B7')-b('B4')))**0.5)",
                               "MSAVI2  = 0.5*(2*b('B8')+1-((2*b('B8')+1)**2-8*(b('B8')-b('B4')))**0.5)",
                             # "MCARI2  = (1.5*(2.5*(b('B7')-b('B4'))-1.3*(b('B7')-b('B3'))))/((((2*b('B7')+1)**2)-(6*b('B7')-5*(b('B4')**0.5))-0.5)**0.5)",
                             # "MTVI2   = (1.5*(1.2*(b('B7')-b('B3'))-2.5*(b('B4')-b('B3'))))/(((2*b('B7')+1)**2-(6*b('B7')-5*b('B4'))-0.5)**0.5)",
                             # "MSR2    = ((b('B7')/b('B4'))-1)/(((b('B7')/b('B4'))+1)**0.5)",
                               "NLI     = ((b('B8')**2)-b('B4'))/((b('B8')**2)+b('B4'))"])

In [4]:
inputImage = alr.format_image(inputImage, inputImage_bands, 'estimateLAI', input_VI_definition)

In [5]:
num_input_pixels = alr.get_num_pixels(inputImage)

In [6]:
scaledImage = alr.scale_image(inputImage, 'estimateLAI')

In [7]:
# names of bands to pass to ee_LARS function (only want Sentinel 2 bands and VIs)
inputBand_names = ['B2', 'B3', 'B4', 'B8', 'GI', 'SGI', 'GVI', 'NDVI3', 'NDVI',
                   'GNDVI', 'NDGI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI2', 'NLI']

In [8]:
select_features = ee.List(alr.ee_LARS(scaledImage, inputBand_names, 'estimateLAI', 5, 10000)).sort()

Selected features:  ['GVI', 'MSR', 'RDVI', 'NDGI', 'B8']


In [9]:
trimmedCollection = alr.trim_data(image=inputImage.updateMask(inputImage.select('estimateLAI').gt(0)),
                                  selected_features=select_features,
                                  response_band='estimateLAI',
                                  num_samples=5000,
                                  num_partitions=10)

exportData = ee.batch.Export.table.toDrive(collection=trimmedCollection,
                                           description='image_samples',
                                           fileFormat='CSV')

exportData.start()

In [10]:
# Essentially a wait loop to see if the data has finished exporting by checking with the server-side
prev_task_status = ee.data.getTaskStatus(exportData.id)[0]["state"]
print(prev_task_status)
while exportData.active():
    task_status = ee.data.getTaskStatus(exportData.id)[0]["state"]
    if(task_status != prev_task_status):
        print(task_status)
    prev_task_status = task_status
    time.sleep(5)
print(ee.data.getTaskStatus(exportData.id)[0]["state"])

READY
RUNNING
COMPLETED


In [11]:
nnet = ee.FeatureCollection("users/hemitshah/nnet5")
nnet_inputs = nnet.filter(ee.Filter.eq("layer_num", 0)).first()
num_inputs = nnet_inputs.getNumber("num_nodes")

selected_features = nnet_inputs.getString("activation").split(",")

nnet = nnet.filterBounds(ee.Geometry.Point([0,0]))
layer_list = nnet.sort("layer_num").toList(nnet.size())

In [12]:
neural_net = layer_list.map(alr.parse_layer)

'''
print(ee.List(neural_net.get(0)).get(0))
print(ee.List(neural_net.get(0)).get(1))
print(ee.List(neural_net.get(1)).get(0))
print(ee.List(neural_net.get(1)).get(1))
print(ee.List(neural_net.get(2)).get(0))
print(ee.List(neural_net.get(2)).get(1))
print(ee.List(neural_net.get(3)).get(0))
print(ee.List(neural_net.get(3)).get(1))
print(ee.List(neural_net.get(4)).get(0))
print(ee.List(neural_net.get(4)).get(1))
'''

'\nprint(ee.List(neural_net.get(0)).get(0))\nprint(ee.List(neural_net.get(0)).get(1))\nprint(ee.List(neural_net.get(1)).get(0))\nprint(ee.List(neural_net.get(1)).get(1))\nprint(ee.List(neural_net.get(2)).get(0))\nprint(ee.List(neural_net.get(2)).get(1))\nprint(ee.List(neural_net.get(3)).get(0))\nprint(ee.List(neural_net.get(3)).get(1))\nprint(ee.List(neural_net.get(4)).get(0))\nprint(ee.List(neural_net.get(4)).get(1))\n'

In [13]:
validation_data = inputImage.select('estimateLAI')
nnet_inputs = scaledImage.select(selected_features)
# layer1out = apply_nnet(neural_net.get(0), nnet_inputs)
# layer2out = apply_nnet(neural_net.get(1), layer1out)
# layer3out = apply_nnet(neural_net.get(2), layer2out)
# layer4out = apply_nnet(neural_net.get(3), layer3out)
# layer5out = apply_nnet(neural_net.get(4), layer4out)
# Map.addLayer(layer1out)
# Map.addLayer(layer2out)
# Map.addLayer(layer3out)
# Map.addLayer(layer4out)
# Map.addLayer(layer5out)

In [14]:
prediction_data = ee.Image(neural_net.iterate(alr.apply_nnet, nnet_inputs)).rename("NNET")
inputsCollection = ee.FeatureCollection("users/hemitshah/image_data_samples").select(selected_features.add("LAI"))

In [15]:
ee_regressor = ee.Classifier.smileRandomForest(numberOfTrees=100,
                                               variablesPerSplit=0,
                                               minLeafPopulation=3,
                                               bagFraction=0.1, seed=0).setOutputMode("REGRESSION")\
                .train(features=inputsCollection, classProperty="LAI", inputProperties=selected_features)

In [16]:
ee_prediction = nnet_inputs.addBands(validation_data).classify(ee_regressor, "RANDOM_FOREST")
# Map.addLayer(validation_data)
# Map.addLayer(prediction_data)
# Map.addLayer(ee_prediction)

In [17]:
nnet_rmse = prediction_data.subtract(validation_data).pow(2).reduceRegion(ee.Reducer.mean(), None, None, None, None, True, 10000000, 1).values().getNumber(0).pow(0.5)
rf_rmse = ee_prediction.subtract(validation_data).pow(2).reduceRegion(ee.Reducer.mean(), None, None, None, None, True, 10000000, 1).values().getNumber(0).pow(0.5)